# Telemetria

Entre as funcionalidades do Istio a telemetria é que materealiza a malha de serviços, ela fornece visibilidade e rastreamento das relações e comunicações na malha.

O interessante é que você obtem a maior partes das funcionalidades gratuitamente por ter instalado o istio-proxy ao lado da sua aplicação, nenhuma configuração adicional será necessária para as funções que iremos explorar. Nas seções seguintes veremos o rastreamento de logs e é a única funcionalidade que necessitará de algumas modificações na nossa aplicação.

## Kiali

Já tivemos contato com o Kiali, agora vamos adicionar as demais ferramentas que completam o conjunto básico de funcionalidades de telemetria, o Jaeger e o Grafana.

Mas antes, vamos voltar ao kiali para explorar em profundidade suas funcionalidades.

Para executá-lo:

In [8]:
istioctl dashboard kiali &
export KIALI_PID=$!

[1] 64613


![Video kiali funcionalidades](media/video-placeholder.jpg)

* Visão geral
* Aplicações (rórulo app=<nome aplicação>)
* Cargas de trabalho (PODs)
* Serviços (Services)
* Gráficos
* Demonstração - bloquear o tráfego utilizando a UI do kiali

> Tipos de leiaute: Há três opções de organização do gráfico para os gráficos do kiali, o padrão - dagre, o 1 - cose bikent e o 2 - cola. A escolha é mais uma opção de estética, mas para conhecer mais sobre os tipos acesse [Cytoscape - Layout extensions](https://js.cytoscape.org/#extensions/layout-extensions).

In [ ]:
# Parando o redirecionamento para o dashboard do kiali
kill $KIALI_PID

## Rastreamento de logs com Jaeger

O rastreamento está integrado com o Istio e Kiali, você deve ter percebido algumas mensagens de erro quando tentavamos acessar funcionalidades de rastreamento.

![kiali erro rastreamento não encontrado](media/kiali-error-tracing.png)

De fato, o Kiali tem um indicador que verifica se os componentes do Istio, jaeger e Grafana estão instalados,

![kiali status](media/kiali-status.png)

Enão, antes de começarmos a utlizar o Jaeger precisaremos instalá-lo em nosso cluster.

In [ ]:
kubectl apply -f istio-1.7.3/samples/addons/jaeger.yaml

Para acessar o dashboard vamos continuar utilizando o `istioctl`, como fizemos para o kiali.

In [9]:
istioctl dashboard jaeger &
export JAEGER_PID=$!

http://localhost:20001/kiali
[2] 65244


![Video Grafana](media/video-placeholder.jpg)

* O que é rastreamento? 
* O que é [OpenTelemetry](https://opentelemetry.io/)
* Instalando o Jaeger
* Visão  geral do Jaeger

Podemos para o processo do `istioctl` para economizar recursos.

In [6]:
# Parando o redirecionamento para o dashboard do Jaeger
kill $JAEGER_PID

bash: kill: (64150) - No such process


: 1

## Replicando cabeçalhos para o rastreamento

A telemetria no Istio é não-envasiva, não precisamos modificar o código para obter a maior parte das funcionalidades, porém para que o rastreamento funcione corretamente precisaremos realizar uma modificação, teremos que replicar o cabeçalho de rastreamento.

Sem essa replicação o Jaeger irá exibir o rastrei apenas da primeira requisição, pois será gerado um novo identificador a cada chamada.

![Jaeger rastreio incompleto](media/jaeger-incomplete-tracing.png)

Infelizmente esse procedimento é invasivo e depende da linguagem de programação. Na documentação do Istio [Trace context propagation](https://istio.io/latest/docs/tasks/observability/distributed-tracing/overview/#trace-context-propagation) há um exemplo de código python e java.

Caso você tenha algum serviço entre as chamadas, como um API Gateway, você poderá utilizá-lo para replicar os cabeçalhos, que são:

* x-request-id
* x-b3-traceid
* x-b3-spanid
* x-b3-parentspanid
* x-b3-sampled
* x-b3-flags
* x-ot-span-context

Na nossa aplicação de demonstração simul-shop o seguinte código em python foi implementado:

```python
from fastapi import FastAPI, Request
from fastapi_opentracing import get_opentracing_span_headers
from fastapi_opentracing.middleware import OpenTracingMiddleware

# App
app = FastAPI()
app.add_middleware(OpenTracingMiddleware)

# service call
def invoke_ws_list(url_list: list, headers = {}):
    logging.debug('Invoked')
    logging.info(f'Calling: {url_list}')
    for url in url_list:
        if url:
            try:
                response = requests.get(url, headers=headers) # <------- Repassa o cabeçalho
                logging.info(f"headers: {response.headers}")
                logging.info(f"body: {response.content}")
            except Exception as err:
                logging.error(f"Failed reaching {url} - {err}")

# Split
@app.get("/s", response_model=MessageOut)
async def split(request: Request):
    carrier = await get_opentracing_span_headers() # <------ Obtem o cabeçalho
    logging.info(f"opentracing: {carrier}")
    logging.info(f"request.headers: {request.headers}")
    url_list = SPLIT_CALL_URL_LST.split(",")
    message = MessageOut(name="split", description=f"List {url_list}")
    invoke_ws_list(url_list, carrier) # <-------- Repassa o cabeçalho
    return message
```

> Código completo está em [exemplos/generic-service/py/app/main.py](exemplos/generic-service/py/app/main.py)

Usando a biblioteca [fastapi_opentracing](https://github.com/wesdu/fastapi-opentracing), obtemos o cabeçalho de rastreamento e o adicionamos no request para o próximo serviço, dessa forma mantêmos a relação entre as requisições.

```log
2020-10-30T21:59:23.841055200Z INFO: 2020-10-30 21:59:23,836 - request.headers: Headers({'host': 'front-end:8000', 'user-agent': 'python-requests/2.24.0', 'accept-encoding': 'gzip, deflate', 'accept': '*/*', 'x-forwarded-proto': 'http', 'x-request-id': '48e17601-e34a-4cc8-b47d-74f666943d14', 'x-envoy-attempt-count': '1', 'content-length': '0', 'x-forwarded-client-cert': 'By=spiffe://cluster.local/ns/default/sa/default;Hash=549a0e87f1c4d64d54a3433318fa1b855b5d45f634477411b8ad84912f06120b;Subject="";URI=spiffe://cluster.local/ns/default/sa/default', 'x-b3-traceid': '5573be4183f426e828c7ecf9e7113264', 'x-b3-spanid': '2449fc64cd15e4eb', 'x-b3-parentspanid': '28c7ecf9e7113264', 'x-b3-sampled': '0'})
```

Exemplo de cabeçalho de rastreamento.

Isso é todo o código que iremos ver para utilizar o Istio, com esse código implementado o nosso rastreamento no Jaeger ficará completo.

![Jaeger rastrei completo](media/jaeger-complete-tracing.png)

E veremos a relação e

## Grafana

![Video grafafa](media/video-placeholder.jpg)

In [ ]:
kubectl apply -f istio-1.7.3/samples/addons/graphana.yaml

In [ ]:
istioctl dashboard graphana &
export GRAPHANA_PID=$!

In [ ]:
# Parando o redirecionamento para o dashboard do graphana
kill $GRAPHANA_PID